In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
import pandas as pd
import numpy as np

import datetime 
import time

%matplotlib inline
import matplotlib.pyplot as plt 
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules

Copied data sets from the Kaggle into data frames  

In [1]:
events_df = pd.read_csv('../input/ecommerce-dataset/events.csv')
category_tree_df = pd.read_csv('../input/ecommerce-dataset/category_tree.csv')
item_properties_1_df = pd.read_csv('../input/ecommerce-dataset/item_properties_part1.csv')
item_properties_2_df = pd.read_csv('../input/ecommerce-dataset/item_properties_part2.csv')

# **Events Data**

In [1]:
events_df

In [1]:
events_df.timestamp = events_df.timestamp.apply(lambda x: datetime.datetime.fromtimestamp(x//1000.0) )

In [1]:
events_df.head()

Visitorid : ID riêng của các user truy cập

Event : Các hành động của người dùng - view ( xem sản phẩm) , add to cart(thêm vào giỏ hàng), transaction ( thực hiện giao dịch)

Itemid : ID của sản phẩm mà user thực hiện hành động

Transactionid : Khi cột này không có dữ liệu thì là nghĩa người dùng chỉ thực hiện view hoặc add to cart

Transactionid : Khi cột này có dữ liệu thì nghĩa là người dùng thực hiện giao dịch

In [1]:
events_df.info()

In [1]:
events_df.describe()

In [1]:
events_df.event.value_counts()

View có           2,664,312 dòng dữ liệu

Addtocart có      69,332

transaction có    22,457

In [1]:
sns.countplot(x='event', data=events_df)

In [1]:
data = events_df.event.value_counts()
labels = data.index
sizes = data.values
explode = (0, 0.15, 0.15)  # explode 1st slice
plt.subplots(figsize=(8,8))
# Plot
plt.pie(sizes, explode=explode, labels=labels,autopct='%1.1f%%', shadow=False, startangle=0)
 
plt.axis('equal')
plt.show()

In [1]:
#Xem số lượng vistort unique
customers  =  events_df.visitorid.unique()
customers.size

In [1]:
# Tổng số lượng visitor
print('Tổng số lượng khách truy cập:', events_df['visitorid'].size)
print('Số khách truy cập thực thế:', customers)

In [1]:
#Những giao dịch được thực hiện
purchased_count  =  len(events_df.transactionid.unique())
print('transaction:',purchased_count)

Ta có số lượng thực hiện hành động transaction là 22,457

Số transaction thực tế là 17,673

Nên ta sẽ biết rằng có những khách hàng thực hiện giao dịch nhiều lần

In [1]:
# Nhóm itemid theo envent
grouped = events_df.groupby('event')['itemid'].apply(list)
grouped

Biểu đồ thể hiện mặt hàng được Xem nhiều nhất

In [1]:
import operator
views = grouped['view']
# creating dictionary for key value pair 
count_view ={}
#since views is a list, we will convert it into numpy array for further manipulations
views = np.array(views[:])
#counting uniques values of views of this numpy views array
unique, counts = np.unique(views, return_counts=True)
# converting unique and counts as a dictionay with key as unique and value as counts
count_view = dict(zip(unique, counts))
#sorting the dictionary
sort_count_view = sorted(count_view.items(), key = operator.itemgetter(1), reverse = True)
# keeping number of unique views on X-axis
x = [i[0] for i in sort_count_view[:7]]
# keeping count number of views on Y-axis
y = [i[1] for i in sort_count_view[:7]]
sns.barplot(x, y, order=x, palette="rocket")


Biểu đồ thể hiện mặt hàng được thêm vào giỏ hàng nhiều nhất



In [1]:
addtocart = grouped['addtocart']
# creating dictionary for key value pair 
count_addtocart ={}
#since addtocart is a list, we will convert it into numpy array for further manipulations
addtocart = np.array(addtocart[:])
#counting uniques values of addtocart items of this numpy addtocart array
unique, counts = np.unique(addtocart, return_counts=True)
# converting unique and counts as a dictionay with key as unique and value as counts
count_addtocart = dict(zip(unique, counts))
#sorting the dictionary
sort_count_addtocart = sorted(count_addtocart.items(), key = operator.itemgetter(1), reverse = True)
# keeping number of unique views on X-axis
x = [i[0] for i in sort_count_addtocart[:7]]
# keeping count number of views on Y-axis
y = [i[1] for i in sort_count_addtocart[:7]]
sns.barplot(x, y, order=x, palette="pastel")

Biểu đồ xem mặt hàng được Mua nhiều nhất



In [1]:
transaction = grouped['transaction']
# creating dictionary for key value pair 
count_transaction ={}
#since addtocart is a list, we will convert it into numpy array for further manipulations
transaction = np.array(transaction[:])
#counting uniques values of addtocart items of this numpy addtocart array
unique, counts = np.unique(transaction, return_counts=True)
# converting unique and counts as a dictionay with key as unique and value as counts
count_transaction = dict(zip(unique, counts))
#sorting the dictionary
sort_count_transaction = sorted(count_transaction.items(), key = operator.itemgetter(1), reverse = True)
# keeping number of unique views on X-axis
x = [i[0] for i in sort_count_transaction[:7]]
# keeping count number of views on Y-axis
y = [i[1] for i in sort_count_transaction[:7]]
sns.barplot(x, y, order=x, palette="vlag")


In [1]:
#Convert date to string
events_df['date'] = [d.date().isoformat() for d in events_df.timestamp]

# Count of visitors each day
count_day = events_df.groupby('date').visitorid.count().to_frame().reset_index()
count_day['date'] = pd.to_datetime(count_day['date'])
count_day.head()

In [1]:
ax = sns.lineplot(x='date', y='visitorid', data=count_day)
ax.set(xlabel="", 
       ylabel="No. of Visitors",
       title="Visitors Over Time")
ax.set_xlim(count_day.iloc[0].date, count_day.iloc[-1].date)
ax.tick_params(axis="x", rotation=45)

In [1]:
temp = events_df.groupby(['date','event']).visitorid.count().to_frame().reset_index()
#
temp['date'] = pd.to_datetime(temp['date'])
temp.head()

In [1]:
ax = sns.lineplot(x='date', y='visitorid', hue='event', data=temp)
ax.set(xlabel="", 
       ylabel="No. of Visitors",
       title="Visitors Over Time by Events")
ax.set_xlim(temp.iloc[0].date, temp.iloc[-1].date)
ax.tick_params(axis="x", rotation=45)

In [1]:
# Những khách hàng thực hiện thêm vào giỏ hàng và giao dịch
temp = events_df[events_df.event != 'view'].groupby(['date','event']).visitorid.count().to_frame().reset_index()
temp['date'] = pd.to_datetime(temp['date'])
temp.head()

In [1]:
ax = sns.lineplot(x='date', y='visitorid', hue='event', data=temp)
ax.set(xlabel="", 
       ylabel="No. of Visitors",
       title="Visitors Over Time by Events")
ax.set_xlim(temp.iloc[0].date, temp.iloc[-1].date)
ax.tick_params(axis="x", rotation=45)

# **Items Data** 
****

In [1]:
item_properties_1_df

In [1]:
item_properties_2_df

In [1]:
items_df = item_properties_1_df.append(item_properties_2_df)

In [1]:
items_df


In [1]:
items_df.timestamp = items_df.timestamp.apply(lambda x: datetime.datetime.fromtimestamp(x//1000.0) )
items_df.head()

Với mỗi itemid thì có một categoryid

Với mỗi iteam thì sẽ tương ứng với các giao dịch của iteam đó( transactionid)



In [1]:
items_df.info()

In [1]:
len(items_df.itemid.unique())

In [1]:
len(items_df.property.unique())

In [1]:
items_df.describe()

Tổng cộng có 20275902 dòng dữ liệu sau khi hết hợp hai file

Có 417053 itemid

Có 1104 property



# **Category Tree**
****

In [1]:
category_tree_df.head()

In [1]:
len(category_tree_df.categoryid.unique())

In [1]:
len(category_tree_df.parentid.unique())

# Làm thêm Bảng Event

Xem nhat ky hoat dong cua khach hang 102019


In [1]:
events_df[events_df.visitorid == 102019].sort_values('timestamp')

**Tạo Bảng Data(date, Số lượng view,SL add tocart, SL transaction,Số lượng visitorid)**

In [1]:
#Gộp bảng dữ liệu theo lượt view
data_event = events_df[events_df.event == 'view'].groupby(['date','event']).visitorid.count().to_frame().reset_index()
data_event['date'] = pd.to_datetime(data_event['date'])
data_event.drop('event', axis=1, inplace=True)
data_event.columns = ['date', 'view']
#Gộp bảng dữ liệu theo lượt addtocart
data_event1 = events_df[events_df.event == 'addtocart'].groupby(['date','event']).visitorid.count().to_frame().reset_index()
data_event1['date'] = pd.to_datetime(data_event1['date'])
data_event1.drop('event', axis=1, inplace=True)
data_event1.columns = ['date', 'addtocart']
#Gộp bảng dữ liệu theo lượt transaction
data_event2 = events_df[events_df.event == 'transaction'].groupby(['date','event']).visitorid.count().to_frame().reset_index()
data_event2['date'] = pd.to_datetime(data_event2['date'])
data_event2.drop('event', axis=1, inplace=True)
data_event2.columns = ['date', 'transaction']
# Nối dữ liệu
data = pd.merge(data_event, data_event1, on='date')
data = pd.merge(data, data_event2, on='date')
data = pd.merge(data, count_day, on='date')

In [1]:
data

In [1]:
#Tạo bảng đếm số lần thực hiện
data_e = events_df.groupby(['visitorid']).event.count().to_frame().reset_index().sort_values('event')
data_e.columns = ['visitorid', 'count_e']
data_e

In [1]:
# Nối vào dữ liệu bảng 
events_df11 = events_df
data_n = pd.merge(events_df11, data_e, on='visitorid')
data_n

In [1]:
# Các khách hàng chỉ vào một lần
data_n1= data_n[data_n['count_e'] == 1]
print(data_n1.count())
print(data_n1.shape)
print('số khách hàng chỉ vào xem 1 lần :',data_n1[data_n1['event'] == 'view'].shape)

Số khách hàng truy cập 1 lần : là 1,001,560

Số khách chỉ xem 1 lần : 998,953

In [1]:
# Xóa dữ liệu các khách hàng chỉ vào xem 1 lần
even_dfn = data_n.drop(data_n[(data_n['event'] == 'view') & (data_n['count_e'] == 1)].index)
print(even_dfn.count())
even_dfn

In [1]:
customers.size

Ta có số lượng khách hàng truy cập unique là 1,407,580

Tổng số lượng giao dịch là 22,457

Số giao dịch unique là 17,673

In [1]:
#Số khách hàng thực hiện giao dịch 
customer_purchased = events_df[events_df.transactionid.notnull()].visitorid.unique()
print("số khách hàng thực hiện giao dịch là", customer_purchased.size)
customer_purchased[:5]

In [1]:
#Xem property là  599528
events_df[events_df.visitorid == 599528].sort_values('visitorid')

In [1]:
#customers

In [1]:
#Những khách hàng thực hiện tìm kiếm mà chưa thực hiện mua là
customer_browsed = []
for x in customers: 
    if x not in customer_purchased:
        customer_browsed.append(x)


In [1]:
print("Số lượng khách hàng tìm kiếm:",len(customer_browsed))
customer_browsed[:5]

In [1]:
# Tạo ra list những sản phẩm mà khách hàng đã đặt mua
# Từ đây có thể dùng hàm để kiểm tra xem sản phẩm nào thường được mua chung với nhau
purchased_items = [] 
for customer in customer_purchased:
    purchased_items.append(list(events_df.loc[(events_df.visitorid == customer) & (events_df.transactionid.notnull())].itemid.values))

In [1]:
print(len(purchased_items))
purchased_items[:2]

In [1]:
def recommender_bought_bought(item_id, purchased_items):
    # chọn ra các sản phẩm đã được mua với sản phảm đã chọn tạo thành list
    recommender_list = []
    for x in purchased_items:
        if item_id in x:
            recommender_list += x
    
    #Bỏ đi sản phẩm đã chọn
    recommender_list = list(set(recommender_list) - set([item_id]))
    
    return recommender_list

In [1]:
# Ta xem thử mặt hàng 15335
recommender_bought_bought(15335, purchased_items)

[15335,  380775,  237753,  317178,  12836,  400969,  105792,  25353,  200793,  80582,  302422]]

Ta thấy các sản phẩm được mua đều nằm trong list sản phẩm recoment

In [1]:
#Xem property là  1046
events_df[events_df.visitorid == 1407576].sort_values('visitorid')


In [1]:
#Tính độ tương quan của các thuộc tính trng bản vửa tạo
corr = data[data.columns].corr()
sns.heatmap(corr,annot = True)

In [1]:

data_trans1 = events_df.groupby(['transactionid']).itemid.count().to_frame().reset_index().sort_values('itemid')
print("Số măt hàng nhiều nhất được mua trong 1 lượt giao dịch:",data_trans1['itemid'].max())
data_trans1

Nhận xét:
* Số lượng khách truy cập là 2756101 ,Số khách truy cập thực thế: 1407580 xấp xỉ một nửa tổng lượng truy cập nên ta có thể đoán là trung bình có thể mỗi khách thực hiện 2 hành động
* Một transaction có thể mua nhiều mặt hàng
* vistorid và view có độ  tương quan cao nhất
* số lượng transaction thì liên quan với addtocart nhiều nhất

# Bảng Items

### groupby dữ liệu của bảng items_df theo property

In [1]:
data_items = items_df.groupby(['property']).itemid.count().to_frame().reset_index()
#data_items[data_items.itemid==1]
data_items

In [1]:
# Kiểm tra xem property nào có nhiều itemid nhất
data_items[data_items.itemid == data_items['itemid'].max()].sort_values('property')

In [1]:
# Kiểm tra xaem property nào có ít itemid nhất
data_items[data_items.itemid == data_items['itemid'].min()].sort_values('property')

In [1]:
#Xem property là  1046
items_df[items_df.property == '1046'].sort_values('itemid')

### groupby dữ liệu của bảng items_df theo itemid

In [1]:

data_items1 = items_df.groupby(['itemid']).property.count().to_frame().reset_index()
data_items1

In [1]:
# Kiểm tra xaem itemid nào nằm trong nhiều property nhất
data_items1[data_items1.property == data_items1['property'].max()].sort_values('itemid')

In [1]:
# Kiểm tra xaem itemid nào nằm trong ít property nhất
data_items1[data_items1.property == data_items1['property'].min()].sort_values('itemid')

In [1]:
#Xem mặt hàng có id là  0
items_df[items_df.itemid == 2].sort_values('property')

In [1]:
data_items1.info()

Kết Luận: 
* Mỗi property có nhiều itemid
* Các itemid có thể trùng nhau nhưng khác property
* propertyid có nhiều itemid nhất là 888 với 3000398 itemid, ít nhất là 1 itemid
* Itemid xuất hiện trong nhiều category nhất 158903 nằm trong 486 categoryid, ít nhất là 207227 và 243157 với 12 categoryid
* Giá trị categoryid là một giá trị được mà hóa
* Giá trị available   = 1 là mặt hàng đang có sẵn, = 0 có nghĩa là đã hết hàng

Nối dữ liệu bẳng iteam, category_tree vs event từ đó tính ra catrgory,parenid nào có lượt view,add tocart, transaction nhiều nhất 


In [1]:
items_df1 = items_df.drop('value', 1)
items_df1.columns = ['itemid', 'categoryid']
items_df1

In [1]:
#items_df2 = pd.merge(items_df1, category_tree_df, on='categoryid')

## Bảng category_tree

In [1]:
category_tree1 = category_tree_df.groupby(['parentid']).categoryid.count().to_frame().reset_index()
category_tree1

In [1]:
# Kiểm tra xaem parentid nào có nhiều categoryid nhất
category_tree1[category_tree1.categoryid == category_tree1['categoryid'].max()].sort_values('parentid')

In [1]:
# Kiểm tra xaem parentid nào có ít categoryid nhất
category_tree1[category_tree1.categoryid == category_tree1['categoryid'].min()].sort_values('parentid')

In [1]:
category_tree2 = category_tree_df.groupby(['categoryid']).parentid.count().to_frame().reset_index()
category_tree2

In [1]:
print("Số lượng parentid nhiều nhất mà iteam có thể có:",category_tree2['parentid'].max())
print("Số lượng parentid ít nhất mà iteam có thể có:",category_tree2['parentid'].min())

Kết luận: 
* Mỗi categoryid chỉ có một parenid
* Mỗi parenid có nhều categoryid
* parenid có nhiều categoryid nhất là 250.0 với 31 
* parenid thì có ít nhất là 1 catgoryid
* categoryid có thể không thuộc về parenid nào